## **Differentiable parameter learning (dPL) + HBV hydrologic model**

## By Multi-scale Hydrology, Processes and Intelligence (MHPI) team from The Pennsylvania State University
## Dr. Chaopeng Shen's Group in Hydrologic Deep Learning and Modeling

## **Citations:**

dHBV1.0: Feng, Dapeng, Liu, Jiangtao, Lawson, Kathryn., & Shen, Chaopeng (2022). Differentiable, learnable, regionalized process-based models with multiphysical outputs can approach state-of-the-art hydrologic prediction accuracy. Water Resources Research, 58, e2022WR032404. https://doi.org/10.1029/2022WR032404

dHBV1.1p: Yalan Song, Kamlesh Sawadekar, Jonathan M Frame, et al. Physics-informed, Differentiable Hydrologic  Models for Capturing Unseen Extreme Events  . ESS Open Archive . March 14, 2025. https://doi.org/10.22541/essoar.172304428.82707157/v2

### **Original Code Release**
Feng, Dapeng, Shen, Chaopeng, Liu, Jiangtao, Lawson, Kathryn, & Beck, Hylke. (2022). differentiable parameter learning (dPL) + HBV hydrologic model. Zenodo. https://doi.org/10.5281/zenodo.7091334

In [ ]:
### Dear Code User,

### Thank you for using our dPL model. We are glad to see our code being
### used to advance research in our field.

### As you use our code, we kindly request that you review and cite
### the relevant papers above that were used to develop the code.
### By doing so, you will help ensure that the contributions of
### the researchers who developed the underlying algorithms
### are properly recognized and appreciated.

### We appreciate your cooperation in this matter and would be happy to
### assist you in finding the appropriate sources to cite.
### If you have any questions or concerns, please do not hesitate to contact us.

### Thank you for your support!
### If you have any question for this release, please contact Dapeng Feng(duf328@psu.edu), Yalan Song (yxs275@psu.edu), or Chaopeng Shen(cshen@engr.psu.edu)

### The code of the differentiable model can be downloaded at https://doi.org/10.5281/zenodo.7091334 or https://github.com/mhpi/dPLHBVrelease
### Reference papers:
### Feng, Dapeng, Jiangtao Liu, Kathryn Lawson, and Chaopeng Shen."Differentiable, Learnable, Regionalized Process‐Based Models With Multiphysical Outputs can Approach State‐Of‐The‐Art Hydrologic Prediction Accuracy." Water Resources Research 58, no. 10 (2022): e2022WR032404.
### Feng, Dapeng, Hylke Beck, Kathryn Lawson, and Chaopeng Shen. "The suitability of differentiable, learnable hydrologic models for ungauged regions and climate change impact assessment." Hydrology and Earth System Sciences (2023). doi: 10.5194/hess-27-2357-2023
### Yalan Song, Kamlesh Sawadekar, Jonathan M Frame, et al. Physics-informed, Differentiable Hydrologic  Models for Capturing Unseen Extreme Events  . ESS Open Archive . March 14, 2025. https://doi.org/10.22541/essoar.172304428.82707157/v2

# The following code is to prepare the input and target data from CAMELS dataset for differentiable hydrologic models.

In [ ]:

"""
Run this code to download CAMELS data locally and get training_file/validation_file which are pickle files containing numpy floats
change rootDatabase = r"E:\Downloads\CAMELS" below to your download directory. This code works on Linux or Windows.
Afterwards, you can simply use the pickle files (upload to colab if running on colab) defined inline below, e.g.,
train_file = 'training_file' # contains train_x, train_y, train_c as tensors
validation_file = 'validation_file' # contains val_x, val_y, val_c
These variables have not been normalized, and are numpy ndarray as follows:
train_x (forcing data, e.g. precipitation, temperature ...): [pixels, time, features]
train_c (constant data, e.g. soil properties, land cover ...): [pixels, features]
train_y (target variable, e.g. soil moisture, streamflow ...): [pixels, time, 1]
val_x, val_c, val_y
The variables to download, training and test time periods, etc., are defined in the last block of this file. Customize as you wish



**simple directions to get set up with Python on your local computer and run this script: 
Download/install Miniconda (mini version of Python package manager, Anaconda)
save this script file in a new folder
optional** download file from https://gdex.ucar.edu/dataset/camels/file/basin_timeseries_v1p2_metForcing_obsFlow.zip and save to folder (otherwise will autodownload via the script but may fail)
open anaconda prompt (miniconda)
navigate to the folder with your script: cd prints current directory, dir prints directory contents, cd foldername moves your current directory into the named folder

then input the following commands into the command line interface (anaconda prompt):
conda create -n test_hydrodl_tutorial python=3.9 ##Important: We are specifying Python 3.9 due to a number of code dependencies and packages that are not yet fully up to date with the most recent versions of python
#(confirm creation with y)
conda activate test_hydrodl_tutorial
conda install requests tqdm pytorch pandas 
python download_CAMELS_script.py

If all goes well, you will be told files "training_file" and "validation_file" were created in subfolder hydroDLpack. 
Upload these files manually to Colab and make sure they are under the folder "hydroDLpack" there, then continue with the tutorial on colab.

"""


In [1]:
from pathlib import Path
import sys
import os
root_path = Path.cwd()
target_path = root_path.parent.parent.parent / 'dPLHBVrelease' / 'hydroDL-dev'

sys.path.append(str(target_path))  # Convert Path object to string before appending

print("Model path:", target_path)

Model path: /data/yxs275/DPL_HBV/Zenodo/dHBV1.1p/dPLHBVrelease/hydroDL-dev


In [37]:
# Where you want your data to be downloaded
rootDatabase = r"/data/yxs275/DPL_HBV/datatest/"
os.chdir(rootDatabase)

In [ ]:
# If you have your CAMELS data downloaded in the path you provided above, you can skip this step! 
import platform

import requests
import zipfile


def download_file(url, destination):
    """Download a file from a specified URL to a given destination."""
    response = requests.get(url, stream=True)
    with open(destination, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

def downloadCAMELS():
  if platform.system() == 'Windows':

    def unzip_file(source, destination):
        """Unzip a file from a source to a destination."""
        with zipfile.ZipFile(source, 'r') as zip_ref:
            zip_ref.extractall(destination)

    # Base directory
    base_dir = os.getcwd()

    # Create necessary directories
    os.makedirs(base_dir + 'camels_attributes_v2.0/', exist_ok=True)
    os.makedirs(base_dir + 'camels_attributes_v2.0/camels_attributes_v2.0/', exist_ok=True)

    # Download and unzip the main dataset
    main_dataset_url = 'https://gdex.ucar.edu/dataset/camels/file/basin_timeseries_v1p2_metForcing_obsFlow.zip'
    main_dataset_dest = base_dir + 'basin_timeseries_v1p2_metForcing_obsFlow.zip'
    download_file(main_dataset_url, main_dataset_dest)
    unzip_file(main_dataset_dest, base_dir + 'basin_timeseries_v1p2_metForcing_obsFlow/')

    ### Download potential evapotranspiration data
    download_file('https://zenodo.org/record/7943626/files/pet_harg.zip?download=1',base_dir + 'pet_harg.zip')
    unzip_file(base_dir + 'pet_harg.zip', base_dir)


    # List of other files to download
    files_to_download = {
        'https://gdex.ucar.edu/dataset/camels/file/camels_attributes_v2.0.xlsx': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_attributes_v2.0.xlsx',
        'https://gdex.ucar.edu/dataset/camels/file/camels_clim.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_clim.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_geol.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_geol.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_hydro.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_hydro.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_name.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_name.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_soil.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_topo.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_topo.txt',
        'https://gdex.ucar.edu/dataset/camels/file/camels_vege.txt': 'camels_attributes_v2.0/camels_attributes_v2.0/camels_vege.txt'
    }

    # Download additional files
    for url, dest in files_to_download.items():
        full_dest = os.path.join(base_dir, dest)
        download_file(url, full_dest)

    print("Download and extraction complete.")

# Add additional download and unzip commands as needed

  else:
    base_dir = os.getcwd()
    print("CAMELS data is downloading to ",base_dir)
    !wget 'https://gdex.ucar.edu/dataset/camels/file/basin_timeseries_v1p2_metForcing_obsFlow.zip' -O {base_dir}'/basin_timeseries_v1p2_metForcing_obsFlow.zip'
    !unzip -o {base_dir}'/basin_timeseries_v1p2_metForcing_obsFlow.zip' -d {base_dir}'/basin_timeseries_v1p2_metForcing_obsFlow'
    !mkdir {base_dir}'/camels_attributes_v2.0/'
    !mkdir {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_attributes_v2.0.xlsx' -O {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/camels_attributes_v2.0.xlsx'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_clim.txt' -O {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/camels_clim.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_geol.txt' -O {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/camels_geol.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_hydro.txt' -O {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/camels_hydro.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_name.txt' -O {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/camels_name.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_soil.txt' -O {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_topo.txt' -O {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/camels_topo.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_soil.txt' -O {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/camels_soil.txt'
    !wget 'https://gdex.ucar.edu/dataset/camels/file/camels_vege.txt' -O {base_dir}'/camels_attributes_v2.0/camels_attributes_v2.0/camels_vege.txt'

    ### Download potential evapotranspiration data
    !wget 'https://zenodo.org/record/7943626/files/pet_harg.zip?download=1' -O {base_dir}'/pet_harg.zip'
    !unzip -o  {base_dir}'/pet_harg.zip' -d {base_dir}

# caution: long-time needed
# you shouldn't need to run this if you are loading directly from pickle file
downloadCAMELS()




CAMELS data is downloading to  /data/yxs275/DPL_HBV/datatest
--2025-06-19 17:41:21--  https://gdex.ucar.edu/dataset/camels/file/basin_timeseries_v1p2_metForcing_obsFlow.zip
Resolving gdex.ucar.edu (gdex.ucar.edu)... 128.117.181.6
Connecting to gdex.ucar.edu (gdex.ucar.edu)|128.117.181.6|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 3406626583 (3.2G) [application/zip]
Saving to: ‘/data/yxs275/DPL_HBV/datatest/basin_timeseries_v1p2_metForcing_obsFlow.zip’

/data/yxs275/DPL_HB 100%[===================>]   3.17G  61.7MB/s    in 51s     

2025-06-19 17:42:13 (63.7 MB/s) - ‘/data/yxs275/DPL_HBV/datatest/basin_timeseries_v1p2_metForcing_obsFlow.zip’ saved [3406626583/3406626583]

unzip:  cannot find or open /content/basin_timeseries_v1p2_metForcing_obsFlow.zip, /content/basin_timeseries_v1p2_metForcing_obsFlow.zip.zip or /content/basin_timeseries_v1p2_metForcing_obsFlow.zip.ZIP.
--2025-06-19 17:42:14--  https://gdex.ucar.edu/dataset/camels/file/camels_attributes_v2.0

In [43]:
# DATA extraction function
from hydroDL import utils
import numpy as np
import pickle
def extractCAMELS(Ttrain,attrLst,varF,camels,forType='daymet',subset_train="All",subset_idx = None,file_path=None):

  train_loader = camels.DataframeCamels(subset=subset_train, tRange=Ttrain, forType=forType)
  x = train_loader.getDataTs(varLst=varF, doNorm=False, rmNan=False)
  y = train_loader.getDataObs(doNorm=False, rmNan=False, basinnorm=False)
  c = train_loader.getDataConst(varLst=attrLst, doNorm=False, rmNan=False)


  # Reading prepared PET data
  # Modify this as the directory where you put PET
  PETDir = camels.dirDB + '/pet_harg/' + forType + '/'
  if subset_train=="All":
     usgsIdLst = camels.gageDict['id']
  else:
     usgsIdLst =  subset_train
  if forType == 'maurer':
      tPETRange = [19800101, 20090101]
  else:
      tPETRange = [19800101, 20150101]
  tPETLst = utils.time.tRange2Array(tPETRange)
  ntime = len(tPETLst)


  PETfull = np.empty([len(usgsIdLst), ntime, 1])
  for k in range(len(usgsIdLst)):
      
      dataTemp = camels.readcsvGage(PETDir, usgsIdLst[k], ['PEVAP'], ntime)
      PETfull[k, :, :] = dataTemp

  TtrainLst = utils.time.tRange2Array(Ttrain)
  C, ind1, ind2 = np.intersect1d(TtrainLst, tPETLst, return_indices=True)
  PETUN = PETfull[:, ind2, :]
  PETUN = PETUN[subset_idx, :, :]
  x = np.concatenate([x, PETUN], axis=2)

  # define dataset
  if file_path is not None:
    with open(file_path, 'wb') as f:
      pickle.dump((x, y, c), f)
  return x, y, c

In [53]:

import os
import hydroDL
from hydroDL.master import default
from hydroDL.data import camels
import numpy as np
import json
# If you already have data extracted in previous runs, set this to True

Justload = False

forType = 'daymet'
Ttrain = [19991001, 20081001] #training period
valid_date = [19861001, 19991001]  # Testing period
#define inputs
if forType == 'daymet':
  varF = [ 'prcp', 'tmean']
else:
  varF = [ 'prcp','tmax']

train_file = 'training_file'
validation_file = 'validation_file'
# Define attributes list
attrLst = [ 'p_mean','pet_mean', 'p_seasonality', 'frac_snow', 'aridity', 'high_prec_freq', 'high_prec_dur',
            'low_prec_freq', 'low_prec_dur', 'elev_mean', 'slope_mean', 'area_gages2', 'frac_forest', 'lai_max',
            'lai_diff', 'gvf_max', 'gvf_diff', 'dom_land_cover_frac', 'dom_land_cover', 'root_depth_50',
            'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity',
            'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'geol_1st_class', 'glim_1st_class_frac',
            'geol_2nd_class', 'glim_2nd_class_frac', 'carbonate_rocks_frac', 'geol_porostiy', 'geol_permeability']

## Load all 671 basins from CAMELS
# TrainLS =  camels.gageDict['id'].tolist()
subsetPath = str(target_path) + '/example/dPLHBV/Sub531ID.txt'
with open(subsetPath, 'r') as fp:
     TrainLS = json.load(fp)
TrainLS.sort()
TrainInd = [TrainLS.index(j) for j in TrainLS]

camels.initcamels(rootDB=rootDatabase)
opt_data = default.optDataCamels # wrapping options around.
opt_data = default.update(opt_data, varT=varF, varC=attrLst, tRange=Ttrain, forType='forType', subset = TrainLS)

if not Justload:
  forcing_train, target_train, attr_train = extractCAMELS(Ttrain,attrLst,varF,camels,forType=forType,subset_train=TrainLS,subset_idx = TrainInd,file_path=train_file)
  forcing_val, target_val, attr_val = extractCAMELS(valid_date,attrLst,varF,camels,forType=forType,subset_train=TrainLS,subset_idx = TrainInd,file_path=validation_file)

  print(f"written files to: "+rootDatabase+os.path.sep+train_file+" and "+validation_file)

daymet tmean was used!
read usgs streamflow 6.449719429016113
read usgs streamflow 6.247171640396118
read usgs streamflow 6.793026685714722
daymet tmean was used!
read usgs streamflow 6.174327373504639
read usgs streamflow 6.168944597244263
read usgs streamflow 6.668954610824585
written files to: /data/yxs275/DPL_HBV/datatest//training_file and validation_file


### You want to know about the $\delta$ HBV model
#### The basic idea of the model is to use an LSTM to generate parameters—either static or dynamic—for the HBV hydrologic model.  
#### The HBV model takes these physical parameters as inputs to simulate streamflow and other internal variables, such as ET, snow water equivalent, and baseflow.  
#### The model can be summarized as:
$$
\begin{align}
\theta_d, \theta_s &= \mathrm{LSTM}(x_{\text{norm}}, A_{\text{norm}}) \\
Q &= \mathrm{HBV}(x, \theta_d, \theta_s)
\end{align}
$$

#### Where \(x\) represents the forcings used to drive the streamflow simulation, including precipitation, mean temperature, and potential evapotranspiration. \(A\) represents the static attributes listed above. 
#### Both \(x\) and \(A\) are concatenated and used as inputs to the LSTM.  Before input, \(x\) and \(A\) need to be normalized. 
#### The HBV model is a process-based model and uses the original (unnormalized) values of \(x\).
#### $\theta_d$ represents the dynamic parameters generated by the sequence-to-sequence LSTM, which change daily.  
#### $\theta_s$ represents the static parameters and uses the values from the last time step of the LSTM prediction.  


### You want to know about HBV parameters:
In $\delta$ HBV1.0 (Feng et al., 2022), $\beta$ (shape coefficient in the soil module) and $\gamma$ (shape coefficient in the evapotranspiration module) are set as dynamic parameters. 
All HBV parameters are: parBETA, parFC, parK0, parK1, parK2, parLP, parPERC, parUZL, parTT, parCFMAX, parCFR, parCWH, parBETAET
The parameter indices of **$\beta$** (`parBETA`) and **$\gamma$** (`parBETAET`) are [1, 13]. All other parameters are set to static to avoid overfitting.

**$\beta$** (shape coefficient in the soil module) and **$\gamma$** (shape coefficient in the evapotranspiration module) are set as dynamic parameters. 

All HBV parameters are: parBETA, parFC, parK0, parK1, parK2, parLP, parPERC, parUZL, parTT, parCFMAX, parCFR, parCWH, parBETAET

The parameter indices of $\beta$ (parBETA) and $\gamma$ (parBETAET) are [1, 13].

In $\delta$ HBV1.1p (Song et al., 2025), All HBV parameters are: parBETA, parFC, parK0, parK1, parK2, parLP, parPERC, parUZL, parTT, parCFMAX, parCFR, parCWH, parBETAET, parC

An additional parameter, `parC`, is introduced to represent upward flow in the HBV model, aiming to improve baseflow simulation.  

More details about the HBV model can be found in the appendix of Song et al. (2025).

For $\delta$HBV1.1p, we consider three options for dynamic parameter combinations:

1. **$\beta$** (`parBETA`) and **$\gamma$** (`parBETAET`), indices **[1, 13]**

2. **$\beta$** (`parBETA`), **$K_0$** (`parK0`), and **$\gamma$** (`parBETAET`), indices **[1, 3, 13]**  
   *Recommended in Song et al. (2025); improves peak flow performance compared to option 1.*

3. **$\beta$** (`parBETA`), **$K_0$** (`parK0`), **$UZL$** (`parUZL`), and **$\gamma$** (`parBETAET`), indices **[1, 3, 8, 13]**  
   *Slight improvement over option 2, but adding more dynamic parameters requires caution to avoid overfitting or instability.*

**$K_0$** and **$UZL$** are parameters that control the generation of near-surface flow (fast flow), and are related to time-dependent connectivity and variable source areas within the watershed.

All other parameters are set to static to avoid overfitting.


## Let's start to train the differentiable model
#### The following code is used to train $\delta$ HBV1.1p  

### Define Statistics Functions and a Scaler
#### These functions are to normalize inputs (x and A) for LSTM

In [54]:
# This part defines a hydrology-specific Scaler class that works similar as sklearn.MinMaxScaler
# getStatDic, calcStat, calcStatgamma are supporting functions
# If you want to use it with your own data, you can create your own scaler that can be used to normalize your data
# Later on, we will use this scaler to transform the train and val data

def getStatDic(log_norm_cols, attrLst=None, attrdata=None, seriesLst=None, seriesdata=None):
  statDict = dict()
  # series data
  if seriesLst is not None:
    for k in range(len(seriesLst)):
      var = seriesLst[k]
      if var in log_norm_cols:
        statDict[var] = calcStatgamma(seriesdata[:, :, k])
      else:
        statDict[var] = calcStat(seriesdata[:, :, k])

  # const attribute
  if attrLst is not None:
    for k in range(len(attrLst)):
      var = attrLst[k]
      statDict[var] = calcStat(attrdata[:, k])
  return statDict

def calcStat(x):
  a = x.flatten()
  b = a[~np.isnan(a)]
  p10 = np.percentile(b, 10).astype(float)
  p90 = np.percentile(b, 90).astype(float)
  mean = np.mean(b).astype(float)
  std = np.std(b).astype(float)
  if std < 0.001:
    std = 1
  return [p10, p90, mean, std]

def calcStatgamma(x):  # for daily streamflow and precipitation
  a = x.flatten()
  b = a[~np.isnan(a)]  # kick out Nan
  b = np.log10(
    np.sqrt(b) + 0.1
  )  # do some tranformation to change gamma characteristics
  p10 = np.percentile(b, 10).astype(float)
  p90 = np.percentile(b, 90).astype(float)
  mean = np.mean(b).astype(float)
  std = np.std(b).astype(float)
  if std < 0.001:
    std = 1
  return [p10, p90, mean, std]

def transNormbyDic( x_in, var_lst, stat_dict, log_norm_cols, to_norm):
  if type(var_lst) is str:
    var_lst = [var_lst]
  x = x_in.copy()
  out = np.full(x.shape, np.nan)
  for k in range(len(var_lst)):
    var = var_lst[k]
    stat = stat_dict[var]
    if to_norm is True:
      if len(x.shape) == 3:
        if var in log_norm_cols:
          x[:, :, k] = np.log10(np.sqrt(x[:, :, k]) + 0.1)
        out[:, :, k] = (x[:, :, k] - stat[2]) / stat[3]
      elif len(x.shape) == 2:
        if var in log_norm_cols:
          x[:, k] = np.log10(np.sqrt(x[:, k]) + 0.1)
        out[:, k] = (x[:, k] - stat[2]) / stat[3]
    else:
      if len(x.shape) == 3:
        out[:, :, k] = x[:, :, k] * stat[3] + stat[2]
        if var in log_norm_cols:
          out[:, :, k] = (np.power(10, out[:, :, k]) - 0.1) ** 2
      elif len(x.shape) == 2:
        out[:, k] = x[:, k] * stat[3] + stat[2]
        if var in log_norm_cols:
          out[:, k] = (np.power(10, out[:, k]) - 0.1) ** 2
  return out



class HydroScaler:
  def __init__(self, attrLst, seriesLst,log_norm_cols):
    self.log_norm_cols = log_norm_cols
    self.attrLst = attrLst
    self.seriesLst = seriesLst
    self.stat_dict = None
    

  def fit(self, attrdata, seriesdata):
    self.stat_dict = getStatDic(
      log_norm_cols=self.log_norm_cols,
      attrLst=self.attrLst,
      attrdata=attrdata,
      seriesLst=self.seriesLst,
      seriesdata=seriesdata,
    )

  def transform(self, data, var_list,):

    norm_data = transNormbyDic(
      data, var_list, self.stat_dict, log_norm_cols = self.log_norm_cols, to_norm=True)

    return norm_data

  def fit_transform(self, attrdata, seriesdata):
    self.fit(attrdata, seriesdata)
    attr_norm = self.transform(attrdata, self.attrLst)
    series_norm = self.transform(seriesdata, self.seriesLst)
    return attr_norm, series_norm


# The function it only used for streamflow unit convert.
def basinNorm(x, basinarea, toNorm):
  nd = len(x.shape)
  if nd == 3 and x.shape[2] == 1:
    x = x[:, :, 0]  # unsqueeze the original 3 dimension matrix
  temparea = np.tile(basinarea, (1, x.shape[1]))
  if toNorm is True:
    flow = (x * 0.0283168 * 3600 * 24) / (temparea * (10 ** 6)) * 10 ** 3  # ft^3/s --> mm/day
  else:

    flow = (
            x
            * ((temparea * (10**6)) * (10 ** (-3)))
            / (0.0283168 * 3600 * 24)
        )  # mm/day -->  ft^3/s
  if nd == 3:
    flow = np.expand_dims(flow, axis=2)
  return flow





### Initialize hydroDL Library, Set Hyperparameters

In [55]:

import torch
import time
import os
from tqdm import trange
import random
from hydroDL import master, utils
from hydroDL.model import crit, train
from hydroDL.model import rnn as rnn
from hydroDL.data import camels
from hydroDL.post import plot, stat
import torch.nn.functional as F
import os
import numpy as np
import torch
from collections import OrderedDict
import random
import pandas as pd
import json
import datetime as dt


traingpuid = 0
torch.cuda.set_device(traingpuid)


# Fix random seed
seedid = 111111
random.seed(seedid)
torch.manual_seed(seedid)
np.random.seed(seedid)
torch.cuda.manual_seed(seedid)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Set hyperparameters
EPOCH = 100 
BATCH_SIZE = 100  # Each batch contrains 100 basins to train the LSTM
RHO = 365   ## The time length of LSTM (RHO + BUFFTIME)
HIDDENSIZE = 256 ## Hidden size of LSTM
saveEPOCH = 10  # save model for every "saveEPOCH" epochs
BUFFTIME = 365  ## The first 365 day is used to warmup (spin-up) the states of HBV model
loadTrain = True


### Please select your model option in the following cell

In [56]:
# Model option: "dHBV1.0" "dHBV1.1p"
Model_option = "dHBV1.1p"
# Dynamic paramter option : for "dHBV1.0", please use [1,13];  for "dHBV1.1p",  you can choose [1,13],[1,3,13] (Yalan recommends this option),[1,3,8, 13]
# If you do not want any dynamic parameters: Set it to an empty list
dyn_option = [1,3,13]

# Number of components: 
# One component means a single set of HBV parameters; 
# 16 components mean 16 sets of HBV parameters, used to run 16 parallel HBV simulations — these are then averaged before the routing step.

# Note: Multiple components do not imply model ensembling using different random seeds. 
# We still use a single neural network to generate all sets of parameters.

# Model performance ranking:
# Static parameters with 1 component (1C) < Static parameters with 16 components (16C)  < Dynamic + static parameters with 16 components (16C)

Nmul = 16


In [57]:
"""
Dataset-specific code to read and scale the data
IF YOU CAN DIRECTLY PROVIDE DATA for train_x, train_y, train_c, val_x, val_y, val_c and scale the data
you can skip this section
The data structure is as follows:
train_x (forcing data, e.g. precipitation, temperature ...), shape: [pixels, time, features]
train_c (constant data, e.g. soil properties, land cover ...), shape: [pixels, features]
target/train_y (e.g. soil moisture, streamflow ...), shape: [pixels, time, 1]
val_x, val_c, val_y
Data type: numpy.float
The Scaler does
attr_norm, series_norm = scaler.fit_transform(train_c, series_data)
val_c = scaler.transform(val_c, attrLst)
val_x = scaler.transform(val_x, varF)
"""

import pickle


# Load X, Y, C from a file
with open(train_file, 'rb') as f:
    forcing_train, target_train, attr_train = pickle.load(f)  # Adjust this line based on your file format
    print('Training forcing dimension:', forcing_train.shape, '(forcing data, e.g. precipitation, temperature ...): [basins, time, features]')
    print('Training attributes dimension:', attr_train.shape, '(constant data, e.g. soil properties, land cover ...): [basins, features]')
    print('Training target dimension:', target_train.shape, '(target variable, e.g. streamflow ...): [basins, time, 1]')
with open(validation_file, 'rb') as g:
    forcing_val, target_val, attr_val= pickle.load(g)  # Adjust this line based on your file format
    print('Testing forcing dimension:', forcing_val.shape)
    print('Testing attributes dimension:', attr_val.shape)
    print('Testing target dimension:', target_val.shape)



if forType == 'daymet':
  varF = [ 'prcp', 'tmean']
else:
  varF = [ 'prcp','tmax']

# Define attributes list
attrLst = [ 'p_mean','pet_mean', 'p_seasonality', 'frac_snow', 'aridity', 'high_prec_freq', 'high_prec_dur',
            'low_prec_freq', 'low_prec_dur', 'elev_mean', 'slope_mean', 'area_gages2', 'frac_forest', 'lai_max',
            'lai_diff', 'gvf_max', 'gvf_diff', 'dom_land_cover_frac', 'dom_land_cover', 'root_depth_50',
            'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity',
            'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'geol_1st_class', 'glim_1st_class_frac',
            'geol_2nd_class', 'glim_2nd_class_frac', 'carbonate_rocks_frac', 'geol_porostiy', 'geol_permeability']

##Add PET in the time series list
varF = varF + ['pet']
# get a Scaler (similar to sklearn.MinMaxScaler).
# basinNorm function is only for converting the streamflow unit from ft^3/s to mm/day.  The basin area are required. y_temp = train_y/(basinarea)
# If your streamflow is not in the unit of ft^3/s, this function needs to be adapted
basinarea  = attr_train[:,np.where(np.array(attrLst)=='area_gages2')[0]]

##Convert the unit of streamflow to mm/day
train_y = basinNorm(target_train,  basinarea, toNorm=True)

val_y = basinNorm(target_val,  basinarea, toNorm=True)

# Initialize scaler
# This scaler will be used to scale the training data and later for the test as well.
# Calculate the statistics for scaling the training data (x and A); Same statistics should be used for validation data for consistency. 
if Model_option == "dHBV1.1p":
    log_norm_cols = []
elif Model_option == "dHBV1.0":
    log_norm_cols = ['prcp']
else:
    raise ValueError("No such model option. Please reselect it.")

scaler = HydroScaler(attrLst=attrLst, seriesLst=varF, log_norm_cols=log_norm_cols)


# Fit and transform training data

attr_norm, series_norm = scaler.fit_transform(attr_train, forcing_train.copy())
attr_norm[np.isnan(attr_norm)] = 0.0   ## Nomalized attributes for LSTM

train_x = forcing_train  # forcing data      ## Input forcing for the HBV model
train_x[np.isnan(train_x)] = 0.0

train_z = series_norm  # normalized forcing data      ## Nomalized forcing for LSTM
train_z[np.isnan(train_z)] = 0.0

train_c = attr_norm


forcTuple_train = (train_x, train_z)

# Fit and transform validation data

val_c = scaler.transform(attr_val, attrLst)
val_z = scaler.transform(forcing_val.copy(), varF)
val_x = forcing_val
# no need to scale val_y before we transform the prediction back to compare with val_y

# fill some gaps in x. A fill for the normalized data is equal to filling with mean values
# We don't fill y because it will be handled by the loss function
val_c[np.isnan(val_c)] = 0.0
val_z[np.isnan(val_z)] = 0.0
val_x[np.isnan(val_x)] = 0.0



#IF YOU CAN DIRECTLY PROVIDE DATA for train_x, train_y, train_c, val_x, val_y, val_c, and a Scaler and scale the data
# you can skip this section
#later code used scaler.stat_dict that matches with this scaler for testing. That could be changed for your data

Training forcing dimension: (531, 3288, 3) (forcing data, e.g. precipitation, temperature ...): [basins, time, features]
Training attributes dimension: (531, 35) (constant data, e.g. soil properties, land cover ...): [basins, features]
Training target dimension: (531, 3288, 1) (target variable, e.g. streamflow ...): [basins, time, 1]
Testing forcing dimension: (531, 4748, 3)
Testing attributes dimension: (531, 35)
Testing target dimension: (531, 4748, 1)


In [58]:
## Train the model
# define loss function
# dHBV1.1p uses a normalized sequare-error-based loss function
if Model_option == "dHBV1.1p":
    optLoss = default.update(default.optLossComb, name='hydroDL.model.crit.NSELossBatch')
    lossFun = crit.NSELossBatch(np.nanstd(train_y, axis=1))
# dHBV1.0 uses a balances RMSE loss
elif Model_option == "dHBV1.0":
    alpha = 0.25 # a weight for RMSE loss to balance low and peak flow
    optLoss = default.update(default.optLossComb, name='hydroDL.model.crit.RmseLossComb', weight=alpha)
    lossFun = crit.RmseLossComb(alpha=alpha)
else:
    raise ValueError("No such model option. Please reselect it.")



In [59]:
TDOpt = True
routing = True # Whether to use the routing module for simulated runoff
comprout = False # True is doing routing for each component
compwts = False # True is using weighted average for components; False is the simple mean
pcorr = None # or a list to give the range of precip correction
dydrop = 0.0 # dropout possibility for those dynamic parameters: 0.0 always dynamic; 1.0 always static
staind = -1 # which time step to use from the learned para time series for those static parameters
ETMod = True
tdRep = dyn_option
tdRepS = [str(ix) for ix in dyn_option]
tdRepS_str = "_".join(tdRepS)
if Model_option == "dHBV1.1p":
    Nfea = 14
# dHBV1.0 uses a balances RMSE loss
elif Model_option == "dHBV1.0":
    Nfea = 13
else:
    raise ValueError("No such model option. Please reselect it.")



rootOut = rootDatabase + f"/dHBV1.1demo/"
if os.path.exists(rootOut) is False:
    os.mkdir(rootOut)
rootOut = rootOut + f"/{Model_option}_{tdRepS_str}/"
    
if os.path.exists(rootOut) is False:
    os.mkdir(rootOut)
print('You model will saved in ',rootOut)

You model will saved in  /data/yxs275/DPL_HBV/datatest//dHBV1.1demo//dHBV1.1p_1_3_13/


In [60]:
# define training options
optTrain = default.update(default.optTrainCamels, miniBatch=[BATCH_SIZE, RHO], nEpoch=EPOCH, saveEpoch=saveEPOCH)
# define output folder to save model results
out = os.path.join(rootOut, f"exp_EPOCH{EPOCH}_BS{BATCH_SIZE}_RHO{RHO}_HS{HIDDENSIZE}_trainBuff{BUFFTIME}_randomseed{seedid}") # output folder to save results
if os.path.exists(out) is False:
    os.mkdir(out)
print('You model will saved in ',out)

You model will saved in  /data/yxs275/DPL_HBV/datatest//dHBV1.1demo//dHBV1.1p_1_3_13/exp_EPOCH100_BS100_RHO365_HS256_trainBuff365_randomseed111111


In [61]:
# define and load model
Ninv = val_x.shape[-1] + val_c.shape[-1]

if Model_option == "dHBV1.1p":
    modelname = 'HBV1_1p'
# dHBV1.0 uses a balances RMSE loss
elif Model_option == "dHBV1.0":
    modelname = 'HBV1_0'
else:
    raise ValueError("No such model option. Please reselect it.")

model = rnn.dHBVModel(ninv=Ninv, nfea=Nfea, nmul=Nmul, hiddeninv=HIDDENSIZE, inittime=BUFFTIME,
                                routOpt=routing, comprout=comprout, compwts=compwts, staind=staind, tdlst=tdRep,
                                dydrop=dydrop, ETMod=ETMod,model_name = modelname)
# dict only for logging
optModel = OrderedDict(name= Model_option, nx=Ninv, nfea=Nfea, nmul=Nmul, hiddenSize=HIDDENSIZE, doReLU=True,
                        Tinv=Ttrain, Trainbuff=BUFFTIME, routOpt=routing, comprout=comprout, compwts=compwts,
                        pcorr=pcorr, staind=staind, tdlst=tdRep, dydrop=dydrop,buffOpt=0, TDOpt=TDOpt, ETMod=ETMod)
# Wrap up all the training configurations to one dictionary in order to save into "out" folder as logging
masterDict = master.wrapMaster(out, opt_data, optModel, optLoss, optTrain)
master.writeMasterFile(masterDict)

write master file /data/yxs275/DPL_HBV/datatest//dHBV1.1demo//dHBV1.1p_1_3_13/exp_EPOCH100_BS100_RHO365_HS256_trainBuff365_randomseed111111/master.json


'/data/yxs275/DPL_HBV/datatest//dHBV1.1demo//dHBV1.1p_1_3_13/exp_EPOCH100_BS100_RHO365_HS256_trainBuff365_randomseed111111'

In [62]:
## Save the statistics 
statFile = os.path.join(out, 'statDict.json')
with open(statFile, 'w') as fp:
    json.dump(scaler.stat_dict, fp, indent=4)
    

In [ ]:
# Train the model

trainedModel = train.trainModel(
    model,
    forcTuple_train,
    train_y,
    train_c,
    lossFun,
    nEpoch=EPOCH,
    miniBatch=[BATCH_SIZE, RHO],
    saveEpoch=saveEPOCH,
    saveFolder=out,
    bufftime=BUFFTIME)

### The model training takes more than 18 hours with GPU for 671 basins and 15 years data.
### Here we download a pre-rained model for testing to save  your time.

## **Let us start to test the model now!**



In [63]:
if Model_option == "dHBV1.1p":
    # Download the dHBV1.1p model from Google Drive
    !gdown 1tWMIy243bxt-r6y4yLFsC8cDcHPTO4BE  # dHBV1.1p model file
    # Copy the downloaded model to the output directory with a new name
    !cp {rootDatabase}/dHBV1_1p_1_3_13_model_Ep100.pt {out}/model_Ep100.pt

elif Model_option == "dHBV1.0":
    # Download the dHBV1.0 model
    !gdown 1qYUMDQyAZjjtwvRShYwLr4GGq5m6DSYC  # dHBV1.0 model file
    # Copy and rename
    !cp {rootDatabase}/dHBV1_0_1_13_model_Ep100.pt {out}/model_Ep100.pt

else:
    raise ValueError("No such model option. Please reselect it.")




Downloading...
From: https://drive.google.com/uc?id=1tWMIy243bxt-r6y4yLFsC8cDcHPTO4BE
To: /data/yxs275/DPL_HBV/datatest/dHBV1_1p_1_3_13_model_Ep100.pt
100%|██████████████████████████████████████| 4.48M/4.48M [00:00<00:00, 56.7MB/s]


In [ ]:
from hydroDL.master import loadModel

print("Load model from ", out)
testepoch = 100
pretrained_model = loadModel(out, epoch=testepoch)
model.inittime = 0
model.lstminv =  pretrained_model.lstminv

val_z_all =val_z #np.concatenate((train_z, val_z),axis = 1) to use the whole training time period as warmup  
val_x_all =val_x #np.concatenate((train_x, val_x),axis = 1) to use the whole training time period as warmup  

val_c_all = np.expand_dims(val_c, axis=1)
val_c_all = np.repeat(val_c_all, val_x_all.shape[1], axis=1)

zTest = np.concatenate([val_z_all, val_c_all], 2)  # Add attributes to historical forcings for LSTM-parameterization

testTuple = (val_x_all, zTest)
testbatch =200

filePathLst = [out+"/Qs",out+"/Q0",out+"/Q1",out+"/Q2",out+"/ET"]
train.testModel(
    model, testTuple, c=None, batchSize=testbatch, filePathLst=filePathLst)

Load model from  /data/yxs275/DPL_HBV/datatest//dHBV1.1demo//dHBV1.1p_1_3_13/exp_EPOCH100_BS100_RHO365_HS256_trainBuff365_randomseed111111


/data/yxs275/DPL_HBV/Zenodo/dHBV1.1p/dPLHBVrelease/hydroDL-dev/hydroDL/model/train.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(modelFile)


batch 0


/data/yxs275/DPL_HBV/Zenodo/dHBV1.1p/dPLHBVrelease/hydroDL-dev/hydroDL/model/rnn.py:321: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  output, hy, cy, reserve, new_weight_buf = torch._cudnn_rnn(


batch 1
batch 2


In [34]:
# This first three year is used to warmup model states.
warmup = pd.date_range(f'{1986}-10-01', f'{1989}-09-30', freq='d')
dataPred = pd.read_csv(  out+"/Qs", dtype=np.float32, header=None).values
dataPred = np.expand_dims(dataPred, axis=-1)


evaDict = [stat.statError(dataPred[:,len(warmup):,0], val_y[:,len(warmup):,0])]
evaDictLst = evaDict
keyLst = ['NSE', 'KGE','FLV','FHV', 'lowRMSE', 'highRMSE']
dataBox = list()
for iS in range(len(keyLst)):
    statStr = keyLst[iS]
    temp = list()
    for k in range(len(evaDictLst)):
        data = evaDictLst[k][statStr]
        #data = data[~np.isnan(data)]
        temp.append(data)
    dataBox.append(temp)


print("dHBV model'NSE', 'KGE','absFLV','absFHV', 'lowRMSE', 'highRMSE'",
      np.nanmedian(dataBox[0][0]),
      np.nanmedian(dataBox[1][0]), np.nanmedian(dataBox[2][0]), np.nanmedian(dataBox[3][0]),
      np.nanmedian(dataBox[4][0]), np.nanmedian(dataBox[5][0]))

/data/yxs275/DPL_HBV/Zenodo/dHBV1.1p/dPLHBVrelease/hydroDL-dev/hydroDL/post/stat.py:60: RuntimeWarning: divide by zero encountered in double_scalars
  PBiaslow[k] = np.sum(lowpred - lowtarget) / np.sum(lowtarget) * 100


dHBV model'NSE', 'KGE','absFLV','absFHV', 'lowRMSE', 'highRMSE' 0.7508290344337746 0.7584913167268741 13.688217778028086 -14.292147055739534 0.06578258920559231 3.309913455796522
